In [4]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import VotingClassifier
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier as cb
import lightgbm as lgb
import mlflow 
import matplotlib.ticker as ticker
#mlflow.set_tracking_uri("./mlruns")
mlflow.set_tracking_uri("http://104.198.187.149:8000")
import geopandas as gpd
from shapely import wkt
from catboost import CatBoostRegressor 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import QuantileTransformer
from optuna.integration import CatBoostPruningCallback



import optuna
from sklearn.metrics import f1_score
pd.options.display.max_columns = None
sklearn.set_config(transform_output='pandas')

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    balanced_accuracy_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
    ConfusionMatrixDisplay
    
)
from dotenv import load_dotenv
import os

load_dotenv("./.env") 
CH_HOST = os.getenv('CH_HOST')
CH_PORT = os.getenv('CH_PORT') 
CH_USER = os.getenv('CH_USER')
CH_PASSWORD = os.getenv('CH_PASSWORD')

import clickhouse_connect

client = clickhouse_connect.get_client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD)

In [2]:
# NYC approximate bounding box
# south, west, north, east
south, west = 40.477399, -74.25909
north, east = 40.917577, -73.700272


In [3]:
import overpy

api = overpy.Overpass()

# build the Overpass QL query
query = f"""
(
  // nodes tagged as cameras
  node["man_made"="camera"]({south},{west},{north},{east});
  node["surveillance"="camera"]({south},{west},{north},{east});
  // ways tagged as cameras
  way["man_made"="camera"]({south},{west},{north},{east});
  way["surveillance"="camera"]({south},{west},{north},{east});
);
out center tags;
"""

# execute
result = api.query(query)

# collect all points
camera_points = []
# nodes
for node in result.nodes:
    camera_points.append({
        "type": "node",
        "lat": float(node.lat),
        "lon": float(node.lon),
        "tags": node.tags
    })
# ways (use .center to get a representative point)
for way in result.ways:
    camera_points.append({
        "type": "way",
        "lat": float(way.center_lat),
        "lon": float(way.center_lon),
        "tags": way.tags
    })

print(f"Found {len(camera_points)} camera objects in NYC.")
for cam in camera_points[:10]:
    print(cam)


Found 4 camera objects in NYC.
{'type': 'node', 'lat': 40.7392384, 'lon': -73.81149, 'tags': {'camera:mount': 'wall', 'camera:type': 'panning', 'man_made': 'surveillance', 'name': 'Herald Mission Center', 'surveillance': 'camera', 'surveillance:type': 'camera', 'surveillance:zone': 'building'}}
{'type': 'node', 'lat': 40.7391523, 'lon': -73.8114182, 'tags': {'camera:mount': 'wall', 'camera:type': 'panning', 'man_made': 'surveillance', 'name': 'Herald Mission Center', 'surveillance': 'camera', 'surveillance:type': 'camera', 'surveillance:zone': 'street'}}
{'type': 'node', 'lat': 40.7764425, 'lon': -73.9110205, 'tags': {'man_made': 'surveillance', 'surveillance': 'camera'}}
{'type': 'way', 'lat': 40.7833727, 'lon': -73.9474504, 'tags': {'addr:housenumber': '1838', 'addr:postcode': '10128', 'addr:street': '2nd Avenue', 'amenity': 'cafe', 'cuisine': 'frozen_yogurt', 'delivery': 'yes', 'dog': 'no', 'level': '0', 'name': 'Go Cups', 'opening_hours': '07:00-22:00', 'payment:american_express': 

In [ ]:
import osmnx as ox

# define bbox as (min_lon, min_lat, max_lon, max_lat)
west, south, east, north = -74.25909, 40.477399, -73.700272, 40.917577
bbox = (west, south, east, north)

tags = {
    "man_made":     "camera",
    "surveillance": "camera",
}

gdf = ox.features_from_bbox(bbox, tags)  

# if you want point locations for ways/polygons:
gdf["geometry"] = gdf.geometry.centroid
points = gdf[gdf.geometry.geom_type == "Point"]

print(f"Found {len(points)} camera points")


Found 4 camera points


/var/tmp/ipykernel_14512/899302535.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["geometry"] = gdf.geometry.centroid


In [2]:
#%pip install selenium webdriver-manager
#!apt-get update
#!apt-get install -y chromium-browser chromium-chromedriver

In [7]:
import requests
import json

# 1) Download the JS file
url = "https://trafficcamphotobooth.com/allCameras.js"
js   = requests.get(url).text

# 2) Find the start of the array literal
kw = "allCameras"
idx = js.find(kw)
if idx == -1:
    raise RuntimeError("allCameras not found in JS")

start = js.find("[", idx)
if start == -1:
    raise RuntimeError("Couldn't find [ after allCameras")

# 3) Walk forward to find the matching closing bracket
depth = 0
for i, ch in enumerate(js[start:], start):
    if   ch == "[": depth += 1
    elif ch == "]":
        depth -= 1
        if depth == 0:
            end = i + 1
            break
else:
    raise RuntimeError("No matching ] found for allCameras array")

array_text = js[start:end]

# 4) Parse it as JSON
cams = json.loads(array_text)

# 5) Pull out lat/lon tuples
coords = [(cam["latitude"], cam["longitude"]) for cam in cams]

print(f"Found {len(coords)} cameras")
print(coords[:5])


Found 922 cameras
[(40.710983, -73.963168), (40.675787, -73.896898), (40.715157, -74.00213), (40.761186, -73.955676), (40.6332, -74.0057)]


In [ ]:
import requests, json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape

# ── 1) get the camera coords ────────────────────────────────────────────────────
js = requests.get("https://trafficcamphotobooth.com/allCameras.js").text
start = js.find("[", js.find("allCameras"))
depth = 0
for i,ch in enumerate(js[start:], start):
    if   ch == "[": depth += 1
    elif ch == "]":
        depth -= 1
        if depth == 0:
            end = i + 1
            break
cams = json.loads(js[start:end])
coords = [(c["latitude"], c["longitude"]) for c in cams]
df = pd.DataFrame(coords, columns=["latitude","longitude"])

# ── 2) fetch the CDTA GeoJSON from Socrata ────────────────────────────────────
# 2020 Community District Tabulation Areas (CDTA), dataset xn3r-zk6y :contentReference[oaicite:0]{index=0}
geojson_url = "https://data.cityofnewyork.us/resource/xn3r-zk6y.geojson?$limit=10000"
resp = requests.get(geojson_url)
resp.raise_for_status()
geo = resp.json()

records = []
for feat in geo["features"]:
    props = feat["properties"]
    cd = props.get("boro_cd")
    geom = shape(feat["geometry"])
    records.append({"community_district": cd, "geometry": geom})
gdf_cd = gpd.GeoDataFrame(records, crs="EPSG:4326")

gdf_pts = gpd.GeoDataFrame(
    df,
    geometry=[Point(lon, lat) for lat,lon in coords],
    crs="EPSG:4326"
)
joined = gpd.sjoin(
    gdf_pts,
    gdf_cd[["community_district","geometry"]],
    how="left",
    predicate="within"
)

out = joined[["latitude","longitude","community_district"]]
out.to_csv("nyc_cameras_with_cd.csv", index=False)
print("Wrote nyc_cameras_with_cd.csv with columns [latitude,longitude,community_district]")


Wrote nyc_cameras_with_cd.csv with columns [latitude,longitude,community_district]


In [8]:
cameras = pd.read_csv("nyc_cameras.csv")
cameras

,latitude,longitude
0,40.710983,-73.963168
1,40.675787,-73.896898
2,40.715157,-74.002130
3,40.761186,-73.955676
4,40.633200,-74.005700
...,...,...
917,40.724006,-73.990992
918,40.717179,-74.012897
919,40.754739,-73.962190
920,40.768025,-73.970282
